<a href="https://colab.research.google.com/github/i2mmmmm/kaggle_flood/blob/main/5.%20ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
os.environ['KAGGLE_USERNAME']='mmmmmlee'
os.environ['KAGGLE_KEY']='b4527f98e27813c6dae3fb199096dcb5'

In [2]:
!kaggle competitions download -c Playground-series-s4e5

 64% 18.0M/28.0M [00:00<00:00, 185MB/s]
100% 28.0M/28.0M [00:00<00:00, 211MB/s]


In [3]:
!unzip 'Playground-series-s4e5.zip'

Archive:  Playground-series-s4e5.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.ensemble import VotingRegressor, BaggingRegressor, StackingRegressor
from sklearn.linear_model import BayesianRidge, SGDRegressor, Ridge
from sklearn.model_selection import cross_val_score, KFold
from sklearn.pipeline import make_pipeline
import xgboost as xgb
import lightgbm as lgb

from keras.models import Sequential
from keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping

In [15]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [16]:
# train 표준화, 스케일링
scaler = StandardScaler()
X = train.drop(columns=['FloodProbability', 'id'])
y = train['FloodProbability']
X_scaled = scaler.fit_transform(X)

In [7]:
# 선형회귀_baseline

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print('R2 Score:', r2_score(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))

R2 Score: 0.8449835489068076
MSE: 0.00040395088979260915


In [13]:
# 다시 feature engineering
train = pd.read_csv('train.csv')

def simplified_getFeats(df):
    num_cols = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality', 'Siltation', 'AgriculturalPractices', 'Encroachments', 'IneffectiveDisasterPreparedness', 'DrainageSystems', 'CoastalVulnerability', 'Landslides', 'Watersheds', 'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss', 'InadequatePlanning', 'PoliticalFactors']

    scaler = StandardScaler().fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])  # Scale early

    BASE_FEATURES = test.columns
    df = train.copy()
    df['CombinedUrbanImpact'] = df['Urbanization'] * df['PopulationScore']
    df['EnvironmentalDegradation'] = df['Deforestation'] + df['Siltation'] + df['WetlandLoss']
    df['InfrastructureVulnerability'] = df['DeterioratingInfrastructure'] + df['DrainageSystems'] + df['DamsQuality']
    df['NaturalDisasterRisk'] = df['MonsoonIntensity'] + df['ClimateChange'] + df['Landslides'] + df['CoastalVulnerability']
    df['ManagementEffectiveness'] = df['RiverManagement'] + df['AgriculturalPractices'] + df['Encroachments'] + df['InadequatePlanning'] + df['PoliticalFactors']
    df['Infrastructure_Risk'] = df['DamsQuality'] * df['DrainageSystems']
    df['wet_Risk'] = df['WetlandLoss'] * df['Encroachments']

    # df['total'] = df[BASE_FEATURES].sum(axis=1)
    df['mean'] = df[BASE_FEATURES].mean(axis=1)
    df['std'] = df[BASE_FEATURES].std(axis=1)
    df['max'] = df[BASE_FEATURES].max(axis=1)
    df['min'] = df[BASE_FEATURES].min(axis=1)
    # df['median'] = df[BASE_FEATURES].median(axis=1)
    # df['ptp'] = df[BASE_FEATURES].values.ptp(axis=1)
    # df['q25'] = df[BASE_FEATURES].quantile(0.25, axis=1)
    # df['q75'] = df[BASE_FEATURES].quantile(0.75, axis=1)
    # df['ClimateImpact'] = df['MonsoonIntensity'] + df['ClimateChange']
    # df['AnthropogenicPressure'] = df['Deforestation'] + df['Urbanization'] + df['AgriculturalPractices'] + df['Encroachments']
    # df['InfrastructureQuality'] = df['DamsQuality'] + df['DrainageSystems'] + df['DeterioratingInfrastructure']
    # df['CoastalVulnerabilityTotal'] = df['CoastalVulnerability'] + df['Landslides']
    # df['PreventiveMeasuresEfficiency'] = df['RiverManagement'] + df['IneffectiveDisasterPreparedness'] + df['InadequatePlanning']
    # df['EcosystemImpact'] = df['WetlandLoss'] + df['Watersheds']
    # df['SocioPoliticalContext'] = df['PopulationScore'] * df['PoliticalFactors']
    return df




train_simp = simplified_getFeats(train)

X = train_simp.drop(columns=['FloodProbability', 'id'])
y = train_simp['FloodProbability']

# models = {
#     'Linear Regression': LinearRegression(),
#     'Ridge Regression': Ridge(),
#     'SGD Regressor': SGDRegressor(),
#     'Bayesian Ridge Regression': BayesianRidge(),
#     'Bagging Regressor': BaggingRegressor(estimator=LinearRegression()),
#     'XGBoost Regressor': xgb.XGBRegressor(),
#     'LightGBM Regressor': lgb.LGBMRegressor(),
# }

# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=35)

# kf = KFold(n_splits=5, shuffle=True, random_state=35)

# for name, model in models.items():
#     mse_values = []
#     r2_values = []
#     for train_index, val_index in kf.split(X):
#         X_train, X_val = X.iloc[train_index], X.iloc[val_index]
#         y_train, y_val = y.iloc[train_index], y.iloc[val_index]

#         model.fit(X_train, y_train)
#         y_pred = model.predict(X_val)
#         r2_values.append(r2_score(y_val, y_pred))
#     avg_r2 = np.mean(r2_values)
#     print(f'{name}: R2 = {avg_r2:.4f}')


In [ ]:
# 다시 feature engineering
train = pd.read_csv('train.csv')

def simplified_getFeats(df):
    num_cols = ['MonsoonIntensity', 'TopographyDrainage', 'RiverManagement', 'Deforestation', 'Urbanization', 'ClimateChange', 'DamsQuality', 'Siltation', 'AgriculturalPractices', 'Encroachments', 'IneffectiveDisasterPreparedness', 'DrainageSystems', 'CoastalVulnerability', 'Landslides', 'Watersheds', 'DeterioratingInfrastructure', 'PopulationScore', 'WetlandLoss', 'InadequatePlanning', 'PoliticalFactors']

    scaler = StandardScaler().fit(df[num_cols])
    df[num_cols] = scaler.transform(df[num_cols])  # Scale early

    # Basic Stats
    df['mean'] = df[num_cols].mean(axis=1)
    df['std'] = df[num_cols].std(axis=1)
    df['max'] = df[num_cols].max(axis=1)
    df['min'] = df[num_cols].min(axis=1)

    # Interaction Features (Simplified)
    df['Climate_Risk'] = df['MonsoonIntensity'] * df['ClimateChange']
    df['Infrastructure_Risk'] = df['DamsQuality'] * df['DrainageSystems']
    df['wet_Risk'] = df['WetlandLoss'] * df['Encroachments']

    return df

train_simp = simplified_getFeats(train)

X = train_simp.drop(columns=['FloodProbability', 'id'])
y = train_simp['FloodProbability']

In [ ]:
def
df['CombinedUrbanImpact'] = df['Urbanization'] * df['PopulationScore']
df['EnvironmentalDegradation'] = df['Deforestation'] + df['Siltation'] + df['WetlandLoss']
df['InfrastructureVulnerability'] = df['DeterioratingInfrastructure'] + df['DrainageSystems'] + df['DamsQuality']
df['NaturalDisasterRisk'] = df['MonsoonIntensity'] + df['ClimateChange'] + df['Landslides'] + df['CoastalVulnerability']
df['ManagementEffectiveness'] = df['RiverManagement'] + df['AgriculturalPractices'] + df['Encroachments'] + df['InadequatePlanning'] + df['PoliticalFactors']

In [17]:
# 훈련, 검증, 테스트 데이터 나누기
X_train, X_temp, y_train, y_temp = train_test_split(X_scaled, y, test_size=0.3, random_state=35)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=35)

# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val), callbacks=[early_stopping])

# 모델 평가
loss, mae = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test MAE:", mae)

Epoch 1/10
24456/24456 [==============================] - 69s 3ms/step - loss: 8.4389e-04 - mae: 0.0185 - val_loss: 3.9723e-04 - val_mae: 0.0151
Epoch 2/10
24456/24456 [==============================] - 61s 2ms/step - loss: 3.7414e-04 - mae: 0.0152 - val_loss: 3.7714e-04 - val_mae: 0.0149
Epoch 3/10
24456/24456 [==============================] - 59s 2ms/step - loss: 3.7144e-04 - mae: 0.0151 - val_loss: 3.6879e-04 - val_mae: 0.0149
Epoch 4/10
24456/24456 [==============================] - 68s 3ms/step - loss: 3.6994e-04 - mae: 0.0151 - val_loss: 3.6329e-04 - val_mae: 0.0149
Epoch 5/10
24456/24456 [==============================] - 71s 3ms/step - loss: 3.6795e-04 - mae: 0.0151 - val_loss: 3.6323e-04 - val_mae: 0.0151
Epoch 6/10
24456/24456 [==============================] - 63s 3ms/step - loss: 3.6696e-04 - mae: 0.0150 - val_loss: 3.6513e-04 - val_mae: 0.0152
Epoch 7/10
24456/24456 [==============================] - 66s 3ms/step - loss: 3.6584e-04 - mae: 0.0150 - val_loss: 3.7413e-04 - v

In [ ]:
# test에도 적용
test = pd.read_csv('test.csv')
test_simp = simplified_getFeats(test)


In [ ]:
# test_simp 데이터셋에서 id를 제외한 특성만 사용하여 예측
X_test_simp = test_simp.drop(columns=['id'])
y_pred_simp = model.predict(X_test_simp)

# 예측 결과를 DataFrame으로 저장
predictions = pd.DataFrame({'id': test_simp['id'], 'PredictedFloodProbability': y_pred_simp.flatten()})

# 예측 결과 출력
print(predictions.head())

23291/23291 [==============================] - 34s 1ms/step
        id  PredictedFloodProbability
0  1117957                   0.577113
1  1117958                   0.455221
2  1117959                   0.450350
3  1117960                   0.469356
4  1117961                   0.469919


In [ ]:
predictions.to_csv('submission_MLP.csv', index=False)

In [ ]:
# test flood 예측 진행

X_train = train_simp.drop(columns=['FloodProbability', 'id'])
y_train = train_simp['FloodProbability']
X_test = test_simp.drop(columns=['id'])

In [ ]:
# 모델 설계
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))  # 출력 뉴런 하나 (홍수 발생 가능성)

# 모델 컴파일
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# 모델 훈련
history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val))

In [ ]:
# LightGBM 모델 학습

lgbm_params = {
    'num_leaves': 183,
    'learning_rate': 0.01183688880802108,
    'n_estimators': 577,
    'subsample_for_bin': 165697,
    'min_child_samples': 114,
    'reg_alpha': 2.075080888948164e-06,
    'reg_lambda': 3.838938366471552e-07,
    'colsample_bytree': 0.9634044234652241,
    'subsample': 0.9592138618622019,
    'max_depth': 9
}

model_lgb = lgb.LGBMRegressor(**lgbm_params)
model_lgb.fit(X_train, y_train)
y_pred_lgb = model_lgb.predict(X_test)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.487907 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1843
[LightGBM] [Info] Number of data points in the train set: 1117957, number of used features: 27
[LightGBM] [Info] Start training from score 0.504480
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gai

In [ ]:
# 출력
test_simp['FloodProbability'] = y_pred_lgb
test_simp[['id', 'FloodProbability']].to_csv('submission_engi_lgb_para.csv', index=False)

# 특성 엔지니어링 이후 제출 - 0.866

In [18]:
sub1 = pd.read_csv('submission_ensemble.csv')
sub2 = pd.read_csv('submission_MLP.csv')
sub3 = pd.read_csv('submission_engi_lgb_para.csv')


In [19]:
print(len(sub1))
print(len(sub2))
print(len(sub3))

745305
745305
745305


In [22]:
sub3.head() #PredictedFloodProbability

,id,FloodProbability
0,1117957,0.577174
1,1117958,0.454428
2,1117959,0.450117
3,1117960,0.466850
4,1117961,0.467364


In [23]:
# 예측값 평균 내기
ensemble_sub = sub1.copy()
ensemble_sub['FloodProbability'] = (sub1['FloodProbability'] + sub2['PredictedFloodProbability'] + sub3['FloodProbability']) / 3

# 앙상블 결과 저장
ensemble_sub.to_csv('submission_ensemble3.csv', index=False)

In [ ]:
ensemble2 = sub2.copy()
ensemble2['FloodProbability'] = sub1['FloodProbability']  +sub1['FloodProbability'] - ensemble_sub['FloodProbability']

In [ ]:
ensemble2.to_csv('submission_ensemble2.csv', index=False)

In [25]:
len(ensemble_sub)

745305

In [ ]:
print(ensemble2.head())

        id  FloodProbability
0  1117957          0.578473
1  1117958          0.455535
2  1117959          0.448232
3  1117960          0.465968
4  1117961          0.466263


In [ ]:
len(ensemble_sub)

745305